In [1]:
from IPython.display import clear_output
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('/home/vco/Projects/pm4py-dcr') # working directory should be pm4py-dcr (the one behind notebooks)
print(os.getcwd())

os.environ['PM4PY_SHOW_EVENT_LOG_DEPRECATION'] = 'false'

/home/vco/Projects/pm4py-dcr/notebooks
/home/vco/Projects/pm4py-dcr


In [2]:
import pm4py
import numpy as np
import pandas as pd
import datetime as dt
import networkx as nx
from pathlib import Path
from copy import deepcopy
from pm4py.util.benchmarking import benchmark_event_log_from_configs
from pm4py.algo.discovery.dcr_discover.algorithm import Variants
from pm4py.objects.dcr import semantics_obj
from datetime import timedelta
from pm4py.algo.evaluation.dcr import algorithm as eval_dcr_on_pdc

pm4py.util.constants.SHOW_EVENT_LOG_DEPRECATION = False
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

# Running example

In [3]:
#test traces for subproceses
dict_keys = ['case:concept:name','concept:name','time:timestamp']

def create_test_traces(ordered_event_matrix):
    start_date = dt.date(2023,1,1)
    case_id = 1
    test_el = []
    for i, trace in enumerate(ordered_event_matrix):
        date = start_date
        for j, event in enumerate(trace):
            test_el.append({dict_keys[0]:str(case_id),dict_keys[1]:str(event),dict_keys[2]:date})
            date = date + dt.timedelta(days=np.random.randint(1, 10)) # TODO: there could be a random sample from a distribution here
        case_id = case_id + 1
    df_test = pd.DataFrame(test_el,columns=['case:concept:name','concept:name','time:timestamp'],dtype=str)
    df_test['time:timestamp'] = pd.to_datetime(df_test['time:timestamp'])
    return pm4py.convert_to_event_log(df_test)

def create_test_traces_with_dates(ordered_event_matrix, ordered_dates_matrix):
    case_id = 1
    test_el = []
    for i, trace in enumerate(ordered_event_matrix):
        for j, event in enumerate(trace):
            date = pd.to_datetime(ordered_dates_matrix[i][j])
            test_el.append({dict_keys[0]:str(case_id),dict_keys[1]:str(event),dict_keys[2]:date})
        case_id = case_id + 1
    df_test = pd.DataFrame(test_el,columns=['case:concept:name','concept:name','time:timestamp'],dtype=str)
    df_test['time:timestamp'] = pd.to_datetime(df_test['time:timestamp'])
    return pm4py.convert_to_event_log(df_test)

A,B,C,D = 'System Choose doctor','Citizen Choose doctor','Family Doctor 1','Family Doctor 2'
E = 'Consent to transfer medical history'
F = 'Transfer medical history'
G = 'Send mail to citizen'
events = [[A,C,G,E,F],
          [A,D,G,E,F],
          [B,E,C,F],
          [B,E,D,F]]
dates = [['2023-01-01','2023-01-08','2023-01-09','2023-01-10','2023-01-11'],
         ['2023-01-01','2023-01-08','2023-01-09','2023-04-01','2023-04-02'],
         ['2023-01-01','2023-01-02','2023-01-04','2023-01-05'],
         ['2023-01-01','2023-01-06','2023-01-07','2023-01-08']]
running_example_trace = create_test_traces_with_dates(events,dates)
tt = pm4py.convert_to_event_log(running_example_trace)
pm4py.write_xes(tt,file_path='/home/vco/Downloads/running_example_trace_doctor.xes')

/home/vco/.pyenv/versions/3.11.0/envs/pm4py-dcr/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
exporting log, completed traces :: 100%|██████████| 4/4 [00:00<00:00, 13497.36it/s]


In [5]:
configs = [{
            'inBetweenRels': False,
            'timed': False,
            'pending': False,
            'variant': Variants.DCR_BASIC,
        },
    {
            'inBetweenRels': True,
            'timed': True,
            'pending': True,
            'variant': Variants.DCR_SUBPROCESS_ME,
        }
]
dcrs = benchmark_event_log_from_configs(tt,dcr_title="running_example",result_file_prefix="running_example_doctor",configs=configs)

[i] Started with config: {'inBetweenRels': False, 'timed': False, 'pending': False, 'variant': <Variants.DCR_BASIC: <module 'pm4py.algo.discovery.dcr_discover.variants.discover_basic' from '/home/vco/Projects/pm4py-dcr/pm4py/algo/discovery/dcr_discover/variants/discover_basic.py'>>}
[i] Mining with basic DisCoveR
[!] Model saved in models/running_example_doctor__config0.xml
[i] Started with config: {'inBetweenRels': True, 'timed': True, 'pending': True, 'variant': <Variants.DCR_SUBPROCESS_ME: <module 'pm4py.algo.discovery.dcr_discover.variants.discover_subprocess_mutual_exclusion' from '/home/vco/Projects/pm4py-dcr/pm4py/algo/discovery/dcr_discover/variants/discover_subprocess_mutual_exclusion.py'>>}
[i] Mining with Sp-DisCoveR (ME)
[!] Model saved in models/running_example_doctor_T_config1.xml


# PDC Challanges 19 - 22

In [309]:
results = eval_dcr_on_pdc.score_everything()

[i] Started for PDC20
[i] Log pdc_2020_1010010.xes
[i] Mining with basic DisCoveR
tp: 516| fp: 38 | tn: 446 | fn: 0
[i] Mining with Sp-DisCoveR (ME)
tp: 516| fp: 35 | tn: 449 | fn: 0
[i] Mining with Sp-DisCoveR (ME)
tp: 516| fp: 39 | tn: 445 | fn: 0
[i] Log pdc_2020_0001000.xes
[i] Mining with basic DisCoveR
tp: 507| fp: 0 | tn: 493 | fn: 0
[i] Mining with Sp-DisCoveR (ME)
tp: 507| fp: 0 | tn: 493 | fn: 0
[i] Mining with Sp-DisCoveR (ME)
tp: 507| fp: 0 | tn: 493 | fn: 0
[i] Log pdc_2020_0001110.xes
[i] Mining with basic DisCoveR


KeyboardInterrupt: 

# Mine from a few BPI logs

In [192]:
from pm4py.objects.dcr.importer import importer as dcr_importer
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
from pm4py.algo.evaluation.simplicity.variants import dcr_relations as dcr_simplicity

In [ ]:
event_log_file = '/home/vco/Datasets/12683249/Road_Traffic_Fine_Management_Process.xes'
dcr_title = 'Traffic Fine'
result_file_prefix = 'rtfmp'
config = [{
            'timed': False,
            'pending': False,
            'variant': Variants.DCR_BASIC
        },{
            'inBetweenRels': True,
            'timed': False,
            'pending': False,
            'variant': Variants.DCR_SUBPROCESS_ME,
}]
dcrs = benchmark_event_log_from_configs(event_log_file,result_file_prefix,dcr_title,config)
rtfmp_reference_dcr = dcrs[0]
rtfmp_sp_dcr = dcrs[1]
rtfmp_manual_dcr = dcr_importer.apply(f'models/{result_file_prefix}_manual_optimization.xml',dcr_importer.Variants.DCRXML)
rtfmp_manual_dcr['subprocesses']['S0'] = {'Appeal to Judge','Insert Date Appeal to Prefecture','Send Appeal to Prefecture','Send Fine'}
rtfmp_manual_dcr['subprocesses']['S1'] = {'S0','Payment'}
rtfmp_manual_dcr['subprocesses']['S2'] = {'Receive Result Appeal from Prefecture','Insert Fine Notification'}
rtfmp_manual_dcr['subprocesses']['S3'] = {'Notify Result Appeal to Offender','Add penalty'}
dcr_simplicity.simplicity_summary('RTFMP',[rtfmp_sp_dcr, rtfmp_manual_dcr],rtfmp_reference_dcr)